In [1]:
#! pip3 install geo2kml
#! pip install pykml
#! pip install geopy

In [2]:
#kml_str = to_kml(geojson)  from geo2kml import to_kml

### Creating expedition diary from csv file with сreated by BaseCamp based on garmin navigator data. (Garmi62S).


1. change columns with time to time 
2. process the data from the description: apply split, indicate the type of station, date, time, if it can be partially extended
3. arrange the columns in order
4. calculate disatnce to first point
6. save to excel

Planned additions

1. output geodata to the map
2. formation of kml file
3. build a graph of relief heights
4. analyze the distance and elevation differences between points



In [3]:
#import librarys
import pandas as pd
from datetime import datetime, timedelta
from geopy.distance import geodesic as GD
#import datetime

In [4]:
# determine the current time
dt = str(datetime.now().date())
dt

'2023-12-05'

In [5]:
# for test you can use "points_from_BaseCamp.csv"
# csv file name input
n=input('Enter name of csv_file fom garmin ', )


Enter name of csv_file fom garmin points_from_BaseCamp.csv


In [6]:
# cut subsidery infromation
df = pd.read_csv(n, sep=',', skiprows=22)
# define the bottom line of useful information
max_ind = df[df['ID'] == 'Address'].index.tolist()[0]-1
max_ind

33

In [7]:
df.head()

,ID,lat,lon,ele,time,magvar,geoidheight,name,cmt,desc,...,ageofdgpsdata,dgpsid,Proximity,Temperature,Depth,DisplayMode,Samples,Expiration,CreationTime,Unnamed: 27
0,1,65.573896989226341,44.625220969319344,0,2023-06-29T10:23:25Z,NaN,NaN,КП1,"cmt: ПСС3 200623 11-28&lt;br&gt;sym: Flag, Blu...","cmt: ПСС3 200623 11-28&lt;br&gt;sym: Flag, Blu...",...,NaN,NaN,NaN,NaN,NaN,SymbolAndName,NaN,NaN,2023-06-29T10:23:25Z,NaN
1,2,65.463922983035445,44.479701993986964,0,2023-06-29T10:23:25Z,NaN,NaN,КП10,"cmt: ПСС4 200623 21-55&lt;br&gt;sym: Flag, Blu...","cmt: ПСС4 200623 21-55&lt;br&gt;sym: Flag, Blu...",...,NaN,NaN,NaN,NaN,NaN,SymbolAndName,NaN,NaN,2023-06-29T10:23:25Z,NaN
2,3,65.456707002595067,44.459505965933204,0,2023-06-29T10:23:25Z,NaN,NaN,КП11,"cmt: ТРИЛ6 200623 22-10&lt;br&gt;sym: Flag, Bl...","cmt: ТРИЛ6 200623 22-10&lt;br&gt;sym: Flag, Bl...",...,NaN,NaN,NaN,NaN,NaN,SymbolAndName,NaN,NaN,2023-06-29T10:23:25Z,NaN
3,4,65.448661968111992,44.434973979368806,0,2023-06-29T10:23:25Z,NaN,NaN,КП12,"cmt: СИЯ 200623 22-30&lt;br&gt;sym: Flag, Blue...","cmt: СИЯ 200623 22-30&lt;br&gt;sym: Flag, Blue...",...,NaN,NaN,NaN,NaN,NaN,SymbolAndName,NaN,NaN,2023-06-29T10:23:25Z,NaN
4,5,65.438431017100811,44.404188003391027,0,2023-06-29T10:23:25Z,NaN,NaN,КП13,"cmt: ПСС3 210623 12-52&lt;br&gt;sym: Flag, Blu...","cmt: ПСС3 210623 12-52&lt;br&gt;sym: Flag, Blu...",...,NaN,NaN,NaN,NaN,NaN,SymbolAndName,NaN,NaN,2023-06-29T10:23:25Z,NaN


In [8]:
#choose only data with information about points
df_clean=df.loc[0:max_ind,['ID','name','lat','lon', 'ele', 'desc','CreationTime']]
df_clean.head()

,ID,name,lat,lon,ele,desc,CreationTime
0,1,КП1,65.573896989226341,44.625220969319344,0,"cmt: ПСС3 200623 11-28&lt;br&gt;sym: Flag, Blu...",2023-06-29T10:23:25Z
1,2,КП10,65.463922983035445,44.479701993986964,0,"cmt: ПСС4 200623 21-55&lt;br&gt;sym: Flag, Blu...",2023-06-29T10:23:25Z
2,3,КП11,65.456707002595067,44.459505965933204,0,"cmt: ТРИЛ6 200623 22-10&lt;br&gt;sym: Flag, Bl...",2023-06-29T10:23:25Z
3,4,КП12,65.448661968111992,44.434973979368806,0,"cmt: СИЯ 200623 22-30&lt;br&gt;sym: Flag, Blue...",2023-06-29T10:23:25Z
4,5,КП13,65.438431017100811,44.404188003391027,0,"cmt: ПСС3 210623 12-52&lt;br&gt;sym: Flag, Blu...",2023-06-29T10:23:25Z


In [9]:
#convert time to DateTime format
df_clean['CreationTime'] = pd.to_datetime(df_clean['CreationTime'])
#convert to float 
df_clean[['lat','lon','ele']] = df_clean[['lat','lon','ele']].astype(float)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   ID            34 non-null     object             
 1   name          34 non-null     object             
 2   lat           34 non-null     float64            
 3   lon           34 non-null     float64            
 4   ele           34 non-null     float64            
 5   desc          34 non-null     object             
 6   CreationTime  34 non-null     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(3), object(3)
memory usage: 2.0+ KB


In [10]:
#clean desc
df_clean.head()

,ID,name,lat,lon,ele,desc,CreationTime
0,1,КП1,65.573897,44.625221,0.0,"cmt: ПСС3 200623 11-28&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
1,2,КП10,65.463923,44.479702,0.0,"cmt: ПСС4 200623 21-55&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
2,3,КП11,65.456707,44.459506,0.0,"cmt: ТРИЛ6 200623 22-10&lt;br&gt;sym: Flag, Bl...",2023-06-29 10:23:25+00:00
3,4,КП12,65.448662,44.434974,0.0,"cmt: СИЯ 200623 22-30&lt;br&gt;sym: Flag, Blue...",2023-06-29 10:23:25+00:00
4,5,КП13,65.438431,44.404188,0.0,"cmt: ПСС3 210623 12-52&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00


In [11]:
df_clean=df_clean.rename(columns={'name':'point_name'})
# create ['station_name','date','time'] name from deesciption
df_clean[['station_name','date','time']] = df_clean['desc'].str.split(pat = ' ', expand = True,n=2)
# create columns duration and distance
df_clean['duration']=3600  
df_clean['distance']=''


In [12]:
# put the columns in the correct order
df_clean=df_clean[['ID','point_name','station_name','date','time','duration','distance', 'lat','lon', 'ele', 'desc','CreationTime']]
df_clean.head(3)

,ID,point_name,station_name,date,time,duration,distance,lat,lon,ele,desc,CreationTime
0,1,КП1,cmt:,ПСС3,"200623 11-28&lt;br&gt;sym: Flag, Blue&lt;br&gt...",3600,,65.573897,44.625221,0.0,"cmt: ПСС3 200623 11-28&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
1,2,КП10,cmt:,ПСС4,"200623 21-55&lt;br&gt;sym: Flag, Blue&lt;br&gt...",3600,,65.463923,44.479702,0.0,"cmt: ПСС4 200623 21-55&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
2,3,КП11,cmt:,ТРИЛ6,"200623 22-10&lt;br&gt;sym: Flag, Blue&lt;br&gt...",3600,,65.456707,44.459506,0.0,"cmt: ТРИЛ6 200623 22-10&lt;br&gt;sym: Flag, Bl...",2023-06-29 10:23:25+00:00


In [13]:
#create columns date
df_clean['date']=df_clean['CreationTime'].map(lambda x: x.strftime("%d.%m.%Y"))
#Create columns time wtih 5 minuts after marked in navigator. This is for guaranted equipment boot.
df_clean['time']=df_clean['CreationTime'].map(lambda x: ((x+timedelta(minutes=5)).strftime("%H:%M")))

df_clean.head()


,ID,point_name,station_name,date,time,duration,distance,lat,lon,ele,desc,CreationTime
0,1,КП1,cmt:,29.06.2023,10:28,3600,,65.573897,44.625221,0.0,"cmt: ПСС3 200623 11-28&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
1,2,КП10,cmt:,29.06.2023,10:28,3600,,65.463923,44.479702,0.0,"cmt: ПСС4 200623 21-55&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
2,3,КП11,cmt:,29.06.2023,10:28,3600,,65.456707,44.459506,0.0,"cmt: ТРИЛ6 200623 22-10&lt;br&gt;sym: Flag, Bl...",2023-06-29 10:23:25+00:00
3,4,КП12,cmt:,29.06.2023,10:28,3600,,65.448662,44.434974,0.0,"cmt: СИЯ 200623 22-30&lt;br&gt;sym: Flag, Blue...",2023-06-29 10:23:25+00:00
4,5,КП13,cmt:,29.06.2023,10:28,3600,,65.438431,44.404188,0.0,"cmt: ПСС3 210623 12-52&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00


In [14]:
# replace work station name to correct station and canal name
stations_dict = {'ПСС1':'PSS1 6E14Z2', 'ПСС2':'PSS2 6C90Z2', 'ПСС3':'PSS3 6E08Z2',  \
                 'ПСС4':'PSS4 6D22Z2', 'ТРИ6':'TRI6 HHZ', 'ТРИ4':'TRI4 HHZ','СИЯ':'SIYA HHZ'}
# apply dictionary with correct station name
df_clean['station_name'] = df_clean['station_name'] .map(stations_dict)

df_clean.head()

,ID,point_name,station_name,date,time,duration,distance,lat,lon,ele,desc,CreationTime
0,1,КП1,NaN,29.06.2023,10:28,3600,,65.573897,44.625221,0.0,"cmt: ПСС3 200623 11-28&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
1,2,КП10,NaN,29.06.2023,10:28,3600,,65.463923,44.479702,0.0,"cmt: ПСС4 200623 21-55&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
2,3,КП11,NaN,29.06.2023,10:28,3600,,65.456707,44.459506,0.0,"cmt: ТРИЛ6 200623 22-10&lt;br&gt;sym: Flag, Bl...",2023-06-29 10:23:25+00:00
3,4,КП12,NaN,29.06.2023,10:28,3600,,65.448662,44.434974,0.0,"cmt: СИЯ 200623 22-30&lt;br&gt;sym: Flag, Blue...",2023-06-29 10:23:25+00:00
4,5,КП13,NaN,29.06.2023,10:28,3600,,65.438431,44.404188,0.0,"cmt: ПСС3 210623 12-52&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00


In [19]:
# sort by latitude
df_clean.sort_values('lat').head()

,ID,point_name,station_name,date,time,duration,distance,lat,lon,ele,desc,CreationTime
26,27,КП35,NaN,29.06.2023,10:28,3600,,65.231668,43.869775,0.0,"cmt: ПСС4 220623 21-55&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
25,26,КП34,NaN,29.06.2023,10:28,3600,,65.248719,43.870663,0.0,"cmt: ПСС3 220623 21-39&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
24,25,КП33,NaN,29.06.2023,10:28,3600,,65.264222,43.878670,0.0,"cmt: СИЯ 220623 16-52&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
23,24,КП32,NaN,29.06.2023,10:28,3600,,65.278402,43.891716,0.0,"cmt: ТРИ6 210623 16-37&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
22,23,КП31,NaN,29.06.2023,10:28,3600,,65.290673,43.897016,0.0,"cmt: ПСС4 210623 16:22&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00


In [20]:
#sort by longitude
df_clean.sort_values('lon').head()

,ID,point_name,station_name,date,time,duration,distance,lat,lon,ele,desc,CreationTime
26,27,КП35,NaN,29.06.2023,10:28,3600,,65.231668,43.869775,0.0,"cmt: ПСС4 220623 21-55&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
25,26,КП34,NaN,29.06.2023,10:28,3600,,65.248719,43.870663,0.0,"cmt: ПСС3 220623 21-39&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
24,25,КП33,NaN,29.06.2023,10:28,3600,,65.264222,43.878670,0.0,"cmt: СИЯ 220623 16-52&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
23,24,КП32,NaN,29.06.2023,10:28,3600,,65.278402,43.891716,0.0,"cmt: ТРИ6 210623 16-37&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
22,23,КП31,NaN,29.06.2023,10:28,3600,,65.290673,43.897016,0.0,"cmt: ПСС4 210623 16:22&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00


In [21]:
#define number points in set
number_points = len(df_clean['point_name'])
number_points

34

In [22]:
# for test you can use "КП35"
input name of first point
first_point_name = input('Enter name of first point ', )
first_ind = df_clean[df_clean['point_name'] == first_point_name].index.tolist()[0]#-1
first_ind

Enter name of first point КП35


26

In [23]:
cord0 = []
#list of first point coordinates 
cord0.append(df_clean.loc[first_ind,'lat'])
cord0.append(df_clean.loc[first_ind,'lon'])
cord0

[65.23166798986495, 43.86977504007518]

In [24]:
#distance calculation function
def diastance1(row):
    # empty lists for coordinates
    cordx = []
    #list of reference point coordinates  
    cordx.append(row['lat'])
    cordx.append(row['lon'])
    #calculate the distance in m
    d=GD(cord0,cordx).m
    return round(d)
    

In [25]:
#df_clean['distance1']=
df_clean['distance']=df_clean.apply(diastance1, axis=1)

In [26]:
df_clean['CreationTime']=df_clean['CreationTime'].astype(str)

In [27]:
df_clean = df_clean.sort_values(by = 'distance')# sort by distance to first point
df_clean.head()

,ID,point_name,station_name,date,time,duration,distance,lat,lon,ele,desc,CreationTime
26,27,КП35,NaN,29.06.2023,10:28,3600,0,65.231668,43.869775,0.0,"cmt: ПСС4 220623 21-55&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
25,26,КП34,NaN,29.06.2023,10:28,3600,1902,65.248719,43.870663,0.0,"cmt: ПСС3 220623 21-39&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
24,25,КП33,NaN,29.06.2023,10:28,3600,3653,65.264222,43.878670,0.0,"cmt: СИЯ 220623 16-52&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
23,24,КП32,NaN,29.06.2023,10:28,3600,5311,65.278402,43.891716,0.0,"cmt: ТРИ6 210623 16-37&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
22,23,КП31,NaN,29.06.2023,10:28,3600,6701,65.290673,43.897016,0.0,"cmt: ПСС4 210623 16:22&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00


In [28]:
#drop "ID" columns
df_clean = df_clean.drop(columns = 'ID', errors = 'ignore')
df_clean.head()

,point_name,station_name,date,time,duration,distance,lat,lon,ele,desc,CreationTime
26,КП35,NaN,29.06.2023,10:28,3600,0,65.231668,43.869775,0.0,"cmt: ПСС4 220623 21-55&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
25,КП34,NaN,29.06.2023,10:28,3600,1902,65.248719,43.870663,0.0,"cmt: ПСС3 220623 21-39&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
24,КП33,NaN,29.06.2023,10:28,3600,3653,65.264222,43.878670,0.0,"cmt: СИЯ 220623 16-52&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
23,КП32,NaN,29.06.2023,10:28,3600,5311,65.278402,43.891716,0.0,"cmt: ТРИ6 210623 16-37&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00
22,КП31,NaN,29.06.2023,10:28,3600,6701,65.290673,43.897016,0.0,"cmt: ПСС4 210623 16:22&lt;br&gt;sym: Flag, Blu...",2023-06-29 10:23:25+00:00


In [29]:
#save dataframe to excel
df_clean.to_excel(r'diary_'+str(n)+'_'+str(number_points)+'points_'+dt+'.xlsx', index= False )